<a href="https://colab.research.google.com/github/OdysseusPolymetis/chr2023/blob/main/4_bert_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def encode_sequence(sequence, tokenizer, model):
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [ ]:
sequence_1 = "I have a pen"
sequence_2 = "This is a pen"
sequence_3 = "I have an apple"

In [ ]:
vec_1 = encode_sequence(sequence_1, tokenizer, model)
vec_2 = encode_sequence(sequence_2, tokenizer, model)
vec_3 = encode_sequence(sequence_3, tokenizer, model)

In [ ]:
similarity_1_2 = cosine_similarity(vec_1.detach().numpy(), vec_2.detach().numpy())[0][0]
similarity_1_3 = cosine_similarity(vec_1.detach().numpy(), vec_3.detach().numpy())[0][0]


similarity_1_2, similarity_1_3

(0.7957161, 0.87849975)

In [ ]:
!gdown 1DjDqd-xn2Hpg-lio12GWuZTjhRwNE5o-
!gdown 1LB3qgsyhNkXcEDYddb_UJsC7STN2BpeA

Downloading...
From: https://drive.google.com/uc?id=1DjDqd-xn2Hpg-lio12GWuZTjhRwNE5o-
To: /content/shakespeare.txt
100% 136k/136k [00:00<00:00, 132MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LB3qgsyhNkXcEDYddb_UJsC7STN2BpeA
To: /content/shaw.txt
100% 130k/130k [00:00<00:00, 115MB/s]


In [ ]:
text_shakespeare = open('shakespeare.txt', encoding='utf8').read()
text_shaw = open('shaw.txt', encoding='utf8').read()

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
sentences_shakespeare = sent_tokenize(text_shakespeare)
sentences_shaw = sent_tokenize(text_shaw)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def encode_sentences_in_batches(sentences, tokenizer, model, batch_size=32):
    all_embeddings = []

    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]

        inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(embeddings)

    all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

In [ ]:
batch_size = 32
encoded_shakespeare = encode_sentences_in_batches(sentences_shakespeare, tokenizer, model, batch_size)
encoded_shaw = encode_sentences_in_batches(sentences_shaw, tokenizer, model, batch_size)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_similarity(encoded_set1, encoded_set2):
    similarities = cosine_similarity(encoded_set1, encoded_set2)
    return similarities


similarities = calculate_similarity(encoded_shakespeare, encoded_shaw)

most_similar_pairs = np.unravel_index(np.argsort(similarities, axis=None)[::-1], similarities.shape)


In [ ]:
for i in range(10):
    shakespeare_idx, shaw_idx = most_similar_pairs[0][i], most_similar_pairs[1][i]
    print(f"Shakespeare phrase: {sentences_shakespeare[shakespeare_idx]}")
    print(f"Shaw phrase: {sentences_shaw[shaw_idx]}")
    print(f"Similarity: {similarities[shakespeare_idx][shaw_idx]}\n")


Shakespeare phrase: What is the Princes Doome?
Shaw phrase: Who is the Angel that cometh?
Similarity: 0.9197890162467957

Shakespeare phrase: What is the Princes Doome?
Shaw phrase: Who is the Angel that cometh?
Similarity: 0.9197890162467957

Shakespeare phrase: What is the Princes Doome?
Shaw phrase: Who is the Angel that cometh?
Similarity: 0.9197889566421509

Shakespeare phrase: What is the Princes Doome?
Shaw phrase: Who is the Angel that cometh?
Similarity: 0.9197889566421509

Shakespeare phrase: How art thou out of breath, when thou hast breth
To say to me, that thou art out of breath?
Shaw phrase: How, if God speaks thy Brother's name,
Dare thou make answer to the call?
Similarity: 0.9182925224304199

Shakespeare phrase: Thou counterfaits a Barke, a Sea, a Wind:
For still thy eyes, which I may call the Sea,
Do ebbe and flow with teares, the Barke thy body is
Sayling in this salt floud, the windes thy sighes,
Who raging with the teares and they with them,
Without a sudden calme 